In [ ]:
import keras.backend as K
from converter.video_converter import VideoConverter

In [ ]:
K.set_learning_phase(0)

# Input/Output resolution
RESOLUTION = 64 # 64x64, 128x128, 256x256
assert (RESOLUTION % 64) == 0, "RESOLUTION should be 64, 128, 256"

In [ ]:
# Architecture configuration
arch_config = {}
arch_config['IMAGE_SHAPE'] = (RESOLUTION, RESOLUTION, 3)
arch_config['use_self_attn'] = True
arch_config['norm'] = "hybrid" # instancenorm, batchnorm, layernorm, groupnorm, none
arch_config['model_capacity'] = "lite" # standard, lite

In [ ]:
from networks.faceswap_gan_model import FaceswapGANModel

In [ ]:
model = FaceswapGANModel(**arch_config)

In [ ]:
model.load_weights(path="./models")

In [ ]:
options = {
    # ===== Fixed =====
    "use_smoothed_bbox": True,
    "use_kalman_filter": True,
    "use_auto_downscaling": False,
    "bbox_moving_avg_coef": 0.65,
    "min_face_area": 35 * 35,
    "IMAGE_SHAPE": model.IMAGE_SHAPE,
    # ===== Tunable =====
    "kf_noise_coef": 3e-3,
    "use_color_correction": "hist_match",
    "detec_threshold": 0.7,
    "roi_coverage": 0.9,
    "enhance": 0.,
    "output_type": 3,
    "direction": "BtoA",
}

In [ ]:
from converter.video_converter import VideoConverter
from face_extractor import FaceExtractor

mtcnn_weights_dir = "./mtcnn_weights/"

minsize=20 
detec_threshold=options["detec_threshold"]
threshold = [0.6, 0.7, detec_threshold]
factor=0.709

extractor = FaceExtractor(min_face_size=minsize, steps_threshold=threshold, 
                          scale_factor=factor)
vc = VideoConverter()

vc.set_face_detector(extractor)
vc.set_gan_model(model)

In [ ]:
input_fn = "videos/trump.mp4"
output_fn = "OUTPUT_VIDEO.mp4"

duration = (10, 20) # None or a non-negative float tuple: (start_sec, end_sec). Duration of input video to be converted

In [ ]:
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)